In [9]:
import csv
import json

# CSV 파일을 읽어서 JSON으로 변환하는 함수
def csv_to_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        
        # CSV 데이터를 JSON으로 변환
        json_data = [row for row in csv_reader]
    
    return json_data

# JSON 데이터를 파일로 저장하는 함수
def save_json_to_file(json_data, output_path):
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(json_data, json_file, ensure_ascii=False, indent=4)

# CSV 파일 경로 설정
input_csv_file = '../응답_데이터.csv'  # 입력 CSV 파일 경로
output_json_file = '응답_데이터.json'  # 출력 JSON 파일 경로

input_csv_file = '../응답_데이터_구술.csv'  # 입력 CSV 파일 경로
output_json_file = '응답_데이터_구술.json'  # 출력 JSON 파일 경로

input_csv_file = '../수험생_능력_추정치.csv'  # 입력 CSV 파일 경로
output_json_file = '수험생_능력_추정치.json'  # 출력 JSON 파일 경로

input_csv_file = '../수험생_능력_추정치_구술.csv'  # 입력 CSV 파일 경로
output_json_file = '수험생_능력_추정치_구술.json'  # 출력 JSON 파일 경로

# CSV를 JSON으로 변환
json_data = csv_to_json(input_csv_file)

# JSON 파일로 저장
save_json_to_file(json_data, output_json_file)

# 변환된 JSON 데이터 출력 (선택 사항)
print(json.dumps(json_data, ensure_ascii=False, indent=4))


[
    {
        "﻿이름": "Gulnara",
        "외국인 등록번호": "3490941780",
        "능력 추정치 (theta)": "-0.7255743411254433"
    },
    {
        "﻿이름": "Dilnoza",
        "외국인 등록번호": "4870107052",
        "능력 추정치 (theta)": "-0.3121240356462392"
    },
    {
        "﻿이름": "Ali",
        "외국인 등록번호": "8426465635",
        "능력 추정치 (theta)": "-1.114182323576095"
    },
    {
        "﻿이름": "Narantuya",
        "외국인 등록번호": "7784311902",
        "능력 추정치 (theta)": "-1.5195127525399874"
    },
    {
        "﻿이름": "Preecha",
        "외국인 등록번호": "5452114233",
        "능력 추정치 (theta)": "0.1627504856037991"
    },
    {
        "﻿이름": "Jose",
        "외국인 등록번호": "9112360080",
        "능력 추정치 (theta)": "-0.8531940419751473"
    },
    {
        "﻿이름": "Liu Yang",
        "외국인 등록번호": "9791149642",
        "능력 추정치 (theta)": "0.6465165275370227"
    },
    {
        "﻿이름": "Bat",
        "외국인 등록번호": "4009665802",
        "능력 추정치 (theta)": "0.9856876754792806"
    },
    {
        "﻿이름": "Wang Fang",
        